### Zutaten zuweisen

In [ ]:
%pip install -U sentence-transformers
%pip install python-Levenshtein
%pip install unidecode

In [ ]:
import pandas as pd
from train_ner import train_model, test_model
from request.Loader.load_annotation import load_dataset
from request.request_gemini import request
from request.request_gpt import chat_with_openai
import pandas as pd
from DataLoader_Ingredients import DataLoader
from enum import Enum
from prompts import ingredients_extraction
import json
import os
import ast
from tqdm import tqdm
import re
from Evaluation.kappa_comparison import evaluate_annotations
from request.recipe_api import request_api
from request.request_modern_bert import Bert
from data_structure.DataType import DataType

from sentence_transformers import SentenceTransformer, util
import numpy as np
from data_structure.model_name import ModelName
from request.Loader.load_normalization import load_response, save_response
from prompts import ingredients_normalization
from data_structure.paths import NOR_EVAL
from request.Loader.load_ner import ner_loader

/Users/noah/Documents/github/MultiAgentBiase/Project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
request("Hello Gemini")

{'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}
API Error: { 'candidates' }


In [3]:
dataset = ner_loader(ModelName.Dataset,DataType.train)

In [4]:
ingredients = []

for item in dataset:
    for key, value in item.items():
        for entity in value.get('entities', []):
            if isinstance(entity, list) and len(entity) == 2 and entity[1] == 'Ingredients':
                ingredients.append(entity[0])

print(len(set(ingredients)))


16995


In [5]:
train = set(ingredients)

In [6]:
prompt = ingredients_normalization.prompt_ingredients_normalization

In [7]:
print(prompt)


    You are a normalization system, to classify Ingredients to their normal ingredients
    you should classify following ingredients
    Ingredients = {$DATA$}
    Here are a few examples:
    haselnusskrokant -> ('haselnusskrokant':'haselnuss'),
    tk-pizza -> ('tk-pizza':'pizza'),
    rot-schwarzesjohannisbeergelee - > ('rot-schwarzesjohannisbeergelee':'johannisbeergelee')
    schweinesteks -> ('schweinesteks':'schweinesteks')
    oliveöl -> ('oliveöl','olivenöl')
    OutputFormat:
    [
        {'LINEOFText':'Normalization'},
        {'LINEOFText':'Normalization'},
    ]
    PLEASE RESPOND ONLY IN THE OUPTPUTFORMAT!
    USE STRICT THE RULES ABOVE!



In [8]:
def pack_twenty(data):
    return [data[i:i+20] for i in range(0, len(data), 20)]

In [9]:
def clean_model_output(str):
    match = re.search(r'\[(.*)\]', str, re.DOTALL)
    if match:
        cleaned_json = "[" + match.group(1).strip() + "]"
    try:
        parsed_data = ast.literal_eval(cleaned_json)
        return parsed_data
    except Exception as e:
        print("Fehler beim Parsen:", e)

In [56]:
request('HelloGemini')

{'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}
API Error: { 'candidates' }


In [11]:
if not os.path.exists(NOR_EVAL/"Gemini.json"):
    packages = pack_twenty(list(train))
    res = []
    for p in packages:
        try:
            new_prompt = prompt.replace("$DATA$", str(p))
            res.extend(clean_model_output(request(new_prompt)))
        except Exception as e:
            print("Model Error:", e)
    save_response(ModelName.Gemini,DataType.eval,res)

##  Pipeline

In [12]:
data =  load_response(ModelName.Gemini,DataType.eval)
data = data[:60]
data

[{'baumnuss': 'walnuss'},
 {'saint - albray': 'saint albray'},
 {'meerrettichwurzel': 'meerrettich'},
 {'tlkreuzkümmel': 'kreuzkümmel'},
 {'currybutter': 'currybutter'},
 {'weisser balsamico-essig': 'balsamicoessig'},
 {'weißwein oder proseco': 'weißwein'},
 {'lachs steak': 'lachssteak'},
 {'frische himbeeren odertk-himbeeren': 'himbeeren'},
 {'basilikumals deko': 'basilikum'},
 {'penne al peperoncino': 'penne'},
 {'fuer die farce': 'farce'},
 {'/braunkäse)': 'braunkäse'},
 {'steinbeisserfilets': 'steinbeisserfilet'},
 {'eierlikör dessert-cremefür': 'eierlikör'},
 {'erdnußöl': 'erdnussöl'},
 {'korianderzweige': 'koriander'},
 {'mini-cocktail-würstchen': 'cocktailwürstchen'},
 {'campignons': 'champignons'},
 {'aprikosenjoghurt': 'aprikosenjoghurt'},
 {'taco maisschalen': 'taco schalen'},
 {'klarer gemüsebrühe': 'gemüsebrühe'},
 {'sahnevanillepudding': 'vanillepudding'},
 {'schokolade - dekorherzen': 'schokolade'},
 {'mandelaroma': 'mandelaroma'},
 {'schoko- bzw. kokosraspeln': 'schokora

In [ ]:
raw_ingredients = [list(obj.keys())[0] for obj in data]

def clean_ingredient(text):
    text = re.sub(r'\d+\s*%', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'tl', '', text)
    text = re.sub(r'tk', '', text)
    text = re.sub(r'[^a-zA-ZäöüÄÖÜß\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()
raw_ingredients = [clean_ingredient(text) for text in raw_ingredients]


In [14]:
def split_multiple_ingredients(item):
    stopwords = ['und', 'oder', 'als',',', 'für', 'bzw','ohne','frische','frisch','kräftiger']
    pattern = r'(?:(?<=\s)(' + '|'.join(stopwords) + r')(?=\s)|(?<=\s)(' + '|'.join(stopwords) + r')|(' + '|'.join(stopwords) + r')(?=\s)|,)'

    parts = re.split(pattern, item)
    buffer = []
    for part in parts:
        if part is None:
            continue
        p = part.strip()
        if p.lower() == 'und':
            buffer.append('&')
        elif p.lower() == 'oder':
            buffer.append('|')
        elif p.lower() == 'ohne':
            buffer.append('(WITHOUT)')
        elif p.lower() in stopwords or p == ',':
            continue
        elif p:
            buffer.append(p)
    return (' '.join(buffer))
ingredients = [split_multiple_ingredients(ing) for ing in raw_ingredients]
ingredients

['baumnuss',
 'saint albray',
 'meerrettichwurzel',
 'kreuzkümmel',
 'currybutter',
 'weisser balsamicoessig',
 'weißwein | proseco',
 'lachs steak',
 'himbeeren | himbeeren',
 'basilikum deko',
 'penne al peperoncino',
 'fuer die farce',
 'braunkäse',
 'steinbeisserfilets',
 'eierlikör dessertcremefür',
 'erdnußöl',
 'korianderzweige',
 'minicocktailwürstchen',
 'campignons',
 'aprikosenjoghurt',
 'taco maisschalen',
 'klarer gemüsebrühe',
 'sahnevanillepudding',
 'schokolade dekorherzen',
 'mandelaroma',
 'schoko kokosraspeln',
 'cabernet sauvignon',
 'hellen teig',
 'granatapfeldressing',
 'ebly zartweizen',
 'baguettebrot',
 'mittelgroße rotepaprikaschote',
 'olivenöl',
 'remulade',
 'salz chilli knoblauchev zwiebelpulver jene die keine zwiebelmögen',
 'fisolen',
 'zimt zucker | puderzucker',
 'neue kartoffeln',
 'schweinekamm | kotelett (WITHOUT) knochen',
 'trinkschokolade',
 'speckwürfeli',
 'grünkohl',
 'creme frech',
 'ganze haselnüsse',
 'zutaten den dip',
 'sauerquark',
 'it

In [36]:
def find_obvious_spices(item):
    spices = ['salz', 'pfeffer', 'zucker', 'zimt', 'majoran']
    pattern = r'\b(' + '|'.join(spices) + r')\b'
    found = re.findall(pattern, item, re.IGNORECASE)
    cleaned = re.sub(pattern, '', item, flags=re.IGNORECASE)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return {
        'original': item,
        'cleaned': cleaned,
        'spices': list({f.lower() for f in found})
    }

ingredients_spices = [find_obvious_spices(ing) for ing in ingredients]
ingredients_spices

[{'original': 'baumnuss', 'cleaned': 'baumnuss', 'spices': []},
 {'original': 'saint albray', 'cleaned': 'saint albray', 'spices': []},
 {'original': 'meerrettichwurzel',
  'cleaned': 'meerrettichwurzel',
  'spices': []},
 {'original': 'kreuzkümmel', 'cleaned': 'kreuzkümmel', 'spices': []},
 {'original': 'currybutter', 'cleaned': 'currybutter', 'spices': []},
 {'original': 'weisser balsamicoessig',
  'cleaned': 'weisser balsamicoessig',
  'spices': []},
 {'original': 'weißwein | proseco',
  'cleaned': 'weißwein | proseco',
  'spices': []},
 {'original': 'lachs steak', 'cleaned': 'lachs steak', 'spices': []},
 {'original': 'himbeeren | himbeeren',
  'cleaned': 'himbeeren | himbeeren',
  'spices': []},
 {'original': 'basilikum deko', 'cleaned': 'basilikum deko', 'spices': []},
 {'original': 'penne al peperoncino',
  'cleaned': 'penne al peperoncino',
  'spices': []},
 {'original': 'fuer die farce', 'cleaned': 'fuer die farce', 'spices': []},
 {'original': 'braunkäse', 'cleaned': 'braunkä